In [1]:
import os

import numpy as np
from skimage.filters import median

from mibidata.mibi_image import MibiImage
from mibidata import tiff

<h2>Introduction</h2>

This notebook provides examples for dealing with MIBI data in the form of a MibiImage instance, which is provided in the [mibitracker-client](https://github.com/ionpath/mibitracker-client/mibidata) library, in the `mibidata` module. Complete documetnation for the [mibidata](https://ionpath.github.io/mibitracker-client/mibidata.html#module-mibidata) module is available.

<h2>Creating a MibiImage Instance</h2>

An instance MibiImage can be created with as little as a numpy array of count data and a list of channel names. Additional args in the constructor allow you to add relevant metadata, including run name and point name. For a full list of metadata args, see the [class documentation](https://ionpath.github.io/mibitracker-client/mibidata.html#module-mibidata.mibi_image). Note that channels can be specified as either a list of integer masses, a list of string channel names, or a list of tuples of (integer mass, string channel name).

In [2]:
# Create some random channel data. MIBI data is typically 16-bit uint, so
# limit range to (0, 65535). Image size here is 128 x 128 pixels with 3
# channels.
random_channel_data = np.random.randint(0, 65535, (128, 128, 3), dtype=np.uint16)
channels = ['Channel 1', 'Channel 2', 'Channel 3']
random_mibi_image = MibiImage(
    random_channel_data, channels,
    run='Random Run', point_name='Random Point Name')
random_mibi_image

An even easier way to import data into a MibiImage instance is to use the [mibidata.tiff.read](https://ionpath.github.io/mibitracker-client/mibidata.html#module-mibidata.tiff) method. `tiff.read` reads in a Multiplexed MIBITIFF file that has been saved in the MIBITIFF format, which is the output of the IONpath TIFF Generator. In this case, all of the metadata is already saved with the image, and so the filename is the only parameter necessary.

In [3]:
image = tiff.read('/path/to/mibi_tiff.tiff')
image

<h2>MibiImage Slicing</h2>

Once loaded or created, the MibiImage class provides conenience methods for slicing, copying, and saving MIBI data. For example, we can select out certain channels of interest from the entire multiplexed image. An image can be sliced using a list of channel names either with the `slice_data` method or indexing.

In [4]:
image.channels

((166, 'beta-tubulin'),
 (144, 'CD11c'),
 (164, 'CD20'),
 (159, 'CD3'),
 (174, 'CD31'),
 (143, 'CD4'),
 (175, 'CD45'),
 (150, 'CD56'),
 (156, 'CD68'),
 (158, 'CD8'),
 (89, 'dsDNA'),
 (146, 'FOXP3'),
 (160, 'HLA class 1 A, B, and C'),
 (176, 'Na-K-ATPase alpha1'),
 (149, 'PD-L1'),
 (151, 'Vimentin'))

In [5]:
channels_of_interest = image.slice_data(['dsDNA', 'beta-tubulin', 'HLA class 1 A, B, and C', 'Na-K-ATPase alpha1'])
np.shape(channels_of_interest)

(1024, 1024, 4)

In [6]:
channels_of_interest = image['dsDNA', 'beta-tubulin', 'HLA class 1 A, B, and C', 'Na-K-ATPase alpha1']
np.shape(channels_of_interest)

(1024, 1024, 4)

If the channel masses are also provided, the masses can be used to slice the image as well.

In [7]:
channels_of_interest = image[89, 166, 160, 176]
np.shape(channels_of_interest)

(1024, 1024, 4)

<h2>Image Manipluation</h2>

A deep copy of MibiImage instances can be made using the `copy` method.

In [8]:
image_copy = image.copy()
image_copy.channels

((166, 'beta-tubulin'),
 (144, 'CD11c'),
 (164, 'CD20'),
 (159, 'CD3'),
 (174, 'CD31'),
 (143, 'CD4'),
 (175, 'CD45'),
 (150, 'CD56'),
 (156, 'CD68'),
 (158, 'CD8'),
 (89, 'dsDNA'),
 (146, 'FOXP3'),
 (160, 'HLA class 1 A, B, and C'),
 (176, 'Na-K-ATPase alpha1'),
 (149, 'PD-L1'),
 (151, 'Vimentin'))

To replace channel data in the image, you will need to access the `.data` property and know the index of the channel in the data array.

In [9]:
for channel in image_copy.channels:
    index = image_copy.channel_inds(channel)
    smoothed = median(image_copy[channel])
    image_copy.data[:, :, index] = smoothed

<h2>Saving MibiImages</h2>

A MibiImage object can be saved as either a multiplexed MIBITIFF or a series of individual channel PNGs.

In [10]:
tiff.write('C:/Users/jayta/Desktop/smoothed.tiff', image_copy)

When saving a series of channel PNGs, the folder to save to must be specified. The filenames of each PNG will be the channel name as it is in the MibiImage object. If the channels are (mass, label) tuples, the filenames will be just the label.

In [11]:
image_copy.export_pngs('/path/to/folder')
outputs = [png_name for png_name in os.path.listdir('/path/to/folder/') if os.path.splitext(png_name)[-1] == '.png']
outputs

['beta-tubulin.png',
 'CD11c.png',
 'CD20.png',
 'CD3.png',
 'CD31.png',
 'CD4.png',
 'CD45.png',
 'CD56.png',
 'CD68.png',
 'CD8.png',
 'dsDNA.png',
 'FOXP3.png',
 'HLA class 1 A, B, and C.png',
 'Na-K-ATPase alpha1.png',
 'PD-L1.png',
 'Vimentin.png']

The exported TIFF or PNGs can then be uploaded to MIBItracker using the `mibitracker` module. See the Python section of the [MIBItracker Tutorial](https://github.com/ionpath/mibitracker-client/blob/master/MIBItracker_API_Tutorial.ipynb) for more information on uploading TIFF or PNG files.